## Simple Collision avoidance

In [ ]:
import numpy as np

# Custom modules

from helpers import global2local, plot_3d_interactive, kill_processes
from simulators import Simulator,Gazebo,QGC


from plan import Plan,State
from gcs import GCS
from simulators.QGroundControl.qgc import QGC
from simulators.gazebo.gazebo import Gazebo
from vehicle_logic import VehicleMode



kill_processes()

## Create Plans

In [2]:
offsets = [(-10, 0, 0, 0),(10, 0, 0, 0)]#
n_vehicles=len(offsets)


homes=np.array([offset[:3] for offset in offsets])
global_paths=[np.array([[-10, 0, 5],[10, 0, 5]]),
              np.array([[10, 0, 5],[-10, 0, 5]])]
local_paths=[global2local(path, home,pairwise=True) for path,home in zip(global_paths,homes)]
plans=[Plan.basic(wps=path,alt=5,wp_margin=0.5,navegation_speed=1) for path in local_paths]

Plan 'basic' created — no connection yet 🧩
Action 'Pre-Arm Check' created — no connection yet 🧩
Step 'Check disarmed' created — no connection yet 🧩
Step 'Check EKF' created — no connection yet 🧩
Step 'Check GPS' created — no connection yet 🧩
Step 'Check system' created — no connection yet 🧩
Action 'Set Mode: GUIDED' created — no connection yet 🧩
Step 'Switch to GUIDED' created — no connection yet 🧩
Action 'Set Navigation Speed' created — no connection yet 🧩
Step 'Set speed to 1.00 m/s' created — no connection yet 🧩
Action 'Arm' created — no connection yet 🧩
Step 'arm' created — no connection yet 🧩
Action 'takeoff' created — no connection yet 🧩
Step 'takeoff' created — no connection yet 🧩
Action 'fly' created — no connection yet 🧩
Step 'go to  -> (0, 0, 5)' created — no connection yet 🧩
Step 'go to  -> (20, 0, 5)' created — no connection yet 🧩
Action 'Land' created — no connection yet 🧩
Step 'land' created — no connection yet 🧩
Plan 'basic' created — no connection yet 🧩
Action 'Pre-Arm 

## Visualization Parameters

In [3]:
waypoints = {f'waypoint_{i+1}': {'pos':global_paths[0],'color':'blue'} for i in range(n_vehicles)}
plot_3d_interactive(waypoints,title='Simulation Markers',expand=[0.2,0.2,0.6],ground=-0.05)

## Choose Simulator

In [ ]:
# simulator = Simulator(name="NONE",offsets=offsets,plans=plans)

# simulator=QGC(offsets=offsets,
#               plans=plans,
#               origin=(-35.3633245,149.1652241,0,0))

simulator=Gazebo(offsets=offsets,
                 plans=plans,
                 world_path='simulators/gazebo/worlds/runway.world',
                 vehicle_models=n_vehicles*['drone'],
                 markers=waypoints)

Launch Simulator

In [5]:
uavs = simulator.launch()

Vehicle 1: Step 'Check disarmed' is now connected ✅🔗
Vehicle 1: Step 'Check EKF' is now connected ✅🔗
Vehicle 1: Step 'Check GPS' is now connected ✅🔗
Vehicle 1: Step 'Check system' is now connected ✅🔗
Vehicle 1: Action 'Pre-Arm Check' is now connected ✅🔗
Vehicle 1: Action 'Pre-Arm Check' is now connected ✅🔗
Vehicle 1: Step 'Switch to GUIDED' is now connected ✅🔗
Vehicle 1: Action 'Set Mode: GUIDED' is now connected ✅🔗
Vehicle 1: Action 'Set Mode: GUIDED' is now connected ✅🔗
Vehicle 1: Step 'Set speed to 1.00 m/s' is now connected ✅🔗
Vehicle 1: Action 'Set Navigation Speed' is now connected ✅🔗
Vehicle 1: Action 'Set Navigation Speed' is now connected ✅🔗
Vehicle 1: Step 'arm' is now connected ✅🔗
Vehicle 1: Action 'Arm' is now connected ✅🔗
Vehicle 1: Action 'Arm' is now connected ✅🔗
Vehicle 1: Step 'takeoff' is now connected ✅🔗
Vehicle 1: Action 'takeoff' is now connected ✅🔗
Vehicle 1: Action 'takeoff' is now connected ✅🔗
Vehicle 1: Step 'go to  -> (0, 0, 5)' is now connected ✅🔗
Vehicle 1: 

## Execute Plan

In [ ]:
avoidance_radius=5

gcs=GCS(positions=homes.copy())
while gcs.incomplete_missions:
    ## Get global positions
    to_remove = set()
    for id in gcs.incomplete_missions:
        uav=uavs[id-1]
        if uav.current_action().name =='fly':
            gcs.ask_position(uav) 
    for id in gcs.incomplete_missions:
        uav = uavs[id-1]
        if uav.current_action().name =='fly':
            jd,min_dist=gcs.get_nearest_neighbor(uav)
            if min_dist<avoidance_radius and uav.get_mode()==VehicleMode.MISSION:  
                gcs.send_neighborg_position(uav,neighborg_id=jd)
                uav.set_mode(VehicleMode.AVOIDANCE)
            if uav.get_mode()==VehicleMode.AVOIDANCE and uav.current_step().state==State.DONE:
                uav.set_mode(VehicleMode.MISSION)
        uav.plan.act()

        if uav.plan.state == State.DONE:
            to_remove.add(id)
    gcs.update_missions(to_remove)
kill_processes()
    

Vehicle 1: ▶️ Starting Step: Check disarmed
Vehicle 2: ▶️ Starting Step: Check disarmed
Vehicle 2: ✅ Step: Check disarmed is done
Vehicle 2: ▶️ Starting Step: Check EKF
Vehicle 1: ✅ Step: Check disarmed is done
Vehicle 1: ▶️ Starting Step: Check EKF
Vehicle 2: ✅ Step: Check EKF is done
Vehicle 2: ▶️ Starting Step: Check GPS
Vehicle 1: ✅ Step: Check EKF is done
Vehicle 1: ▶️ Starting Step: Check GPS
Vehicle 2: ✅ Step: Check GPS is done
Vehicle 2: ▶️ Starting Step: Check system
Vehicle 1: ✅ Step: Check GPS is done
Vehicle 1: ▶️ Starting Step: Check system
Vehicle 2: ✅ Step: Check system is done
Vehicle 2: ▶️ Starting Step: Switch to GUIDED
Vehicle 2: ✅ Step: Switch to GUIDED is done
Vehicle 2: ▶️ Starting Step: Set speed to 1.00 m/s
Vehicle 2: ✅ Step: Set speed to 1.00 m/s is done
Vehicle 2: ▶️ Starting Step: arm
Vehicle 1: ✅ Step: Check system is done
Vehicle 1: ▶️ Starting Step: Switch to GUIDED
Vehicle 1: ✅ Step: Switch to GUIDED is done
Vehicle 1: ▶️ Starting Step: Set speed to 1.00 

In [7]:
uavs[0].plan

✅ <Plan 'basic'>
  ✅ <Action 'Pre-Arm Check'>
    ✅ <Step 'Check disarmed'>
    ✅ <Step 'Check EKF'>
    ✅ <Step 'Check GPS'>
    ✅ <Step 'Check system'>
  ✅ <Action 'Set Mode: GUIDED'>
    ✅ <Step 'Switch to GUIDED'>
  ✅ <Action 'Set Navigation Speed'>
    ✅ <Step 'Set speed to 1.00 m/s'>
  ✅ <Action 'Arm'>
    ✅ <Step 'arm'>
  ✅ <Action 'takeoff'>
    ✅ <Step 'takeoff'>
  ✅ <Action 'fly'>
    ✅ <Step 'go to  -> (0, 0, 5)'>
    ✅ <Step 'go to (avoidance) -> (7.494907174052721, 1.022558499869061, 4.911520481109619)'>
    ✅ <Step 'go to (avoidance) -> (8.161230319407874, 1.644080939294109, 4.901727199554443)'>
    ✅ <Step 'go to (avoidance) -> (8.68028308155196, 2.3274621332567174, 4.89987850189209)'>
    ✅ <Step 'go to (avoidance) -> (9.674227567561397, 2.3779403668952774, 4.898726940155029)'>
    ✅ <Step 'go to (avoidance) -> (10.147967053389415, 2.612511164613907, 4.894093036651611)'>
    ✅ <Step 'go to  -> (20, 0, 5)'>
  ✅ <Action 'Land'>
    ✅ <Step 'land'>

In [8]:
uavs[1].plan

✅ <Plan 'basic'>
  ✅ <Action 'Pre-Arm Check'>
    ✅ <Step 'Check disarmed'>
    ✅ <Step 'Check EKF'>
    ✅ <Step 'Check GPS'>
    ✅ <Step 'Check system'>
  ✅ <Action 'Set Mode: GUIDED'>
    ✅ <Step 'Switch to GUIDED'>
  ✅ <Action 'Set Navigation Speed'>
    ✅ <Step 'Set speed to 1.00 m/s'>
  ✅ <Action 'Arm'>
    ✅ <Step 'arm'>
  ✅ <Action 'takeoff'>
    ✅ <Step 'takeoff'>
  ✅ <Action 'fly'>
    ✅ <Step 'go to  -> (0, 0, 5)'>
    ✅ <Step 'go to (avoidance) -> (-7.5370247697253285, -1.0208916336261755, 4.9045491218566895)'>
    ✅ <Step 'go to (avoidance) -> (-8.208015231187467, -1.661803237573756, 4.903050422668457)'>
    ✅ <Step 'go to (avoidance) -> (-8.697042000314166, -2.370529707757755, 4.8955206871032715)'>
    ✅ <Step 'go to (avoidance) -> (-9.66554750101311, -2.436239059085688, 4.893253803253174)'>
    ✅ <Step 'go to (avoidance) -> (-10.126634937163196, -2.7161256104177465, 4.890469074249268)'>
    ✅ <Step 'go to  -> (-20, 0, 5)'>
  ✅ <Action 'Land'>
    ✅ <Step 'land'>